In [24]:
import cv2
from scipy.stats import entropy as scipy_entropy
from PIL import Image
from skimage.transform import resize
from skimage.io import imread
from skimage.color import rgb2gray
import skimage.measure        
from numpy import unique

import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

path_1 = '/content/img1.jpg'
path_2 = '/content/img2.jpg'
path_3 = '/content/img.png'

In [23]:
def resizing(img_path):

    input = cv2.imread(img_path)

    image = cv2.resize(input, (128,128)) 

    return(image)

In [7]:
def quantizer(clusters, img_path, task):
    """Quantizes an image based on a colour palette that K-means creates
    and saves the quantized image on the disk
    Args:
        clusters (int): The number of the clusters i.e. the number of colours
        img_path (str): The path of the input image
    """
    # Load the image
    # image = cv2.imread(img_path)
    image = resizing(img_path)
    h, w = image.shape[:2]

    # Conversion to L*a*b* gamut for perptual meaning
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # reshape the image into a feature vector for k-means
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    # Apply K-means and create the quantized image
    clf = MiniBatchKMeans(n_clusters = clusters)
    labels = clf.fit_predict(image)
    quantized = clf.cluster_centers_.astype("uint8")[labels]

    # Reshape the feature vectors to back to images
    quantized = quantized.reshape((h, w, 3))
    image = image.reshape((h, w, 3))

    # Convert from L*a*b* to RGB
    quantized = cv2.cvtColor(quantized, cv2.COLOR_LAB2RGB, cv2.CV_8U)
    image = cv2.cvtColor(image, cv2.COLOR_LAB2RGB, cv2.CV_8U)

    if task == 'preprocessing':
      return quantized

    if task == 'see_preprocessed_img':
      plt.imshow(np.hstack([image, quantized]))
      plt.show()
      
    if task == 'save_preprocessed_img':
      quantized = cv2.cvtColor(quantized, cv2.COLOR_RGB2BGR, cv2.CV_8U)
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR, cv2.CV_8U)
      cv2.imwrite(f"color_quantization_{clusters}.jpg", np.hstack([image, quantized]))     

In [4]:
def entropy(n_colors, img_path):

    image = quantizer(n_colors, img_path, 'preprocessing')

    value, counts = unique(image, return_counts=True)
    entropy =  scipy_entropy(counts, base=2)

    # return(print(f"Image Entropy scipy {entropy}"))
    return entropy

In [5]:
def shannon_entropy(n_colors, img_path):

    image = quantizer(n_colors, img_path, 'preprocessing')

    entropy = skimage.measure.shannon_entropy(image)

    # return(print(f"Image Entropy skimage {entropy}"))
    return entropy

In [31]:
# colors = 25
# quantizer(colors, path_1, 'see_preprocessed_img')
# colors = 60
# quantizer(colors, path_2, 'see_preprocessed_img')
# colors = 20
# quantizer(colors, path_3, 'see_preprocessed_img')
# entropy(colors, path_2)